In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# import general necessary packages
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os, cv2

In [ ]:
sum()

In [ ]:
# import the necessary packages -- CNN_Regression
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from numpy.random import seed
from numpy import savez_compressed
from keras.callbacks import EarlyStopping

In [ ]:
# define images data file path (When we define the path, we can use the value word to do next things such as save or load)
path = "/content/drive/My Drive/Spring Deep Learning Class material /Week 3/Weather project /grasped Image/"
train_dir = path + "Train/"
val_dir = path + "Validation/"
test_dir = path + "Test/"
result = path + "Model Results/"

In [ ]:
'''
# load image train data
x_train_images = []
# make sure to read it in order
for image_path in sorted(os.listdir(train_dir)):
  img_path = train_dir + image_path
  image = cv2.imread(img_path)
  x_train_images.append(image)
x_train = np.array(x_train_images)'''

'\n# load image train data\nx_train_images = []\n# make sure to read it in order\nfor image_path in sorted(os.listdir(train_dir)):\n  img_path = train_dir + image_path\n  image = cv2.imread(img_path)\n  x_train_images.append(image)\nx_train = np.array(x_train_images)'

In [ ]:
'''# load image validation data
x_val_images = []
# make sure to read it in order
for image_path in sorted(os.listdir(val_dir)):
  img_path = val_dir + image_path
  image = cv2.imread(img_path)
  x_val_images.append(image)
x_val = np.array(x_val_images)'''

'# load image validation data\nx_val_images = []\n# make sure to read it in order\nfor image_path in sorted(os.listdir(val_dir)):\n  img_path = val_dir + image_path\n  image = cv2.imread(img_path)\n  x_val_images.append(image)\nx_val = np.array(x_val_images)'

In [ ]:
'''# load image test data
x_test_images = []
# make sure to read it in order
for image_path in sorted(os.listdir(test_dir)):
  img_path = test_dir + image_path
  image = cv2.imread(img_path)
  x_test_images.append(image)
x_test = np.array(x_test_images)'''

'# load image test data\nx_test_images = []\n# make sure to read it in order\nfor image_path in sorted(os.listdir(test_dir)):\n  img_path = test_dir + image_path\n  image = cv2.imread(img_path)\n  x_test_images.append(image)\nx_test = np.array(x_test_images)'

In [ ]:
'''# save loaded and converted image data 
savez_compressed(path+'train_data.npz', x_train)
savez_compressed(path+'val_data.npz', x_val)
savez_compressed(path+'test_data.npz1', x_test)'''

"# save loaded and converted image data \nsavez_compressed(path+'train_data.npz', x_train)\nsavez_compressed(path+'val_data.npz', x_val)\nsavez_compressed(path+'test_data.npz1', x_test)"

In [ ]:
# load image numpy array from npz file
from numpy import load
# load dict of arrays
train_data = load(path+'train_data.npz')
x_train = train_data['arr_0']
val_data = load(path+'val_data.npz')
x_val = val_data['arr_0']
test_data = load(path+'test_data.npz')
x_test = test_data['arr_0']

In [ ]:
# import weather data
target_data = pd.read_csv("/content/drive/My Drive/Spring Deep Learning Class material /Week 3/Weather project /grasped Image/model_ready.csv")
target_data.head()

,valid,tmpf,drct,sknt
0,2016-09-01 00:00:00,75.02,222.500000,5.125000
1,2016-09-01 01:00:00,73.94,202.857143,4.857143
2,2016-09-01 02:00:00,73.94,218.181818,4.272727
3,2016-09-01 03:00:00,73.94,204.444444,4.555556
4,2016-09-01 04:00:00,73.04,145.000000,2.700000


In [ ]:
# convert to a timedate
target_data['valid'] =  pd.to_datetime(target_data['valid'])
# set timedate to index
target_data = target_data.set_index('valid')
target_data.head()

,tmpf,drct,sknt
valid,,,
2016-09-01 00:00:00,75.02,222.500000,5.125000
2016-09-01 01:00:00,73.94,202.857143,4.857143
2016-09-01 02:00:00,73.94,218.181818,4.272727
2016-09-01 03:00:00,73.94,204.444444,4.555556
2016-09-01 04:00:00,73.04,145.000000,2.700000


In [ ]:
target_data.isna().sum()

tmpf    21
drct     7
sknt     3
dtype: int64

In [ ]:
target_data[target_data.sknt == 0]

,tmpf,drct,sknt
valid,,,
2016-09-01 15:00:00,69.98,0.0,0.0
2016-09-04 00:00:00,62.96,0.0,0.0
2016-09-10 06:00:00,73.04,0.0,0.0
2016-09-12 09:00:00,51.08,0.0,0.0
2016-09-13 06:00:00,55.04,0.0,0.0
...,...,...,...
2017-08-24 10:00:00,57.90,0.0,0.0
2017-08-26 10:00:00,51.10,0.0,0.0
2017-08-28 04:00:00,55.00,0.0,0.0


In [ ]:
# we decide to fill all missing value to 0
target_data = target_data.fillna(0)
target_data.head()

,tmpf,drct,sknt
valid,,,
2016-09-01 00:00:00,75.02,222.500000,5.125000
2016-09-01 01:00:00,73.94,202.857143,4.857143
2016-09-01 02:00:00,73.94,218.181818,4.272727
2016-09-01 03:00:00,73.94,204.444444,4.555556
2016-09-01 04:00:00,73.04,145.000000,2.700000


In [ ]:
# fill the 0 with the past 5 hous mean
# sknt column
 for i in range(0,8719):
  if target_data.iloc[i][2] == 0: # 2 represent the coulumn name. which is equal to 'sknt'. the fist i is means the row number. so this two number is corresponded.
    target_data['sknt'].iloc[i] = total = target_data['sknt'].iloc[i-5:i].mean()

In [ ]:
target_data[target_data.sknt == 0] # no 0s anymore

,tmpf,drct,sknt
valid,,,


In [ ]:
# set train and test dataset
a = pd.Timestamp('2017/06/01 00:00:00')
b = pd.Timestamp('2017/08/01 00:00:00')
sample = target_data[target_data.index < b]
train = target_data[target_data.index < a]
val = sample[sample.index >= a]
test = target_data[target_data.index >= b]

In [ ]:
# separate the data into train, validation and test
# tmpf model
y_train_3 = train['sknt']
y_val_3 = val['sknt']
y_test_3 = test['sknt']

NameError: ignored

In [ ]:
print("Image train dataset shape: ", x_train.shape, "Weather train dataset shape: ", train.shape)
print("Image validation dataset shape: ", x_val.shape, "Weather validation dataset shape: ", val.shape)
print("Image test dataset shape: ", x_test.shape, "Weather test dataset shape: ", test.shape)

NameError: ignored

In [ ]:
# Model 2 ( we don't need tf.keras ast all)
model2 = tf.keras.Sequential()

model2.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', input_shape=(125,145,3))) 
model2.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model2.add(tf.keras.layers.Dropout(0.3))

model2.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu')) # filters refer to what?
model2.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model2.add(tf.keras.layers.Dropout(0.3))

model2.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model2.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model2.add(tf.keras.layers.Dropout(0.3))

model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(512, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(512, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(256, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(128, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(1, activation='linear'))

In [ ]:
#opt = Adam(lr=1e-3, decay=1e-3 / 200)
model2.compile(optimizer='rmsprop',loss="mse", metrics=['mae'])

In [ ]:
es2 = EarlyStopping(monitor='val_loss', mode='min', 
                   # LOOK HOW QUICK IT STOPS IF YOU DON'T USE
                   # PATIENCE - SUBJECT TO NOISE!!!
                   # will vary based on every problem you tackle
                   # try a patience of 2, 5, 10, 50, 100...
                   # what do you notice?
                   patience=10, 
                   verbose=1)

In [ ]:
history2 = model2.fit(x_train, y_train_3, validation_data=(x_val, y_val_3), epochs=500, batch_size=5,verbose=1,callbacks=[es2])

In [ ]:
# let's try to extract from history.history
x = np.array(history2.epoch)
x = max(x)
print("We should re-run for this many epochs: ", x)

In [ ]:
# let's see the training and validation loss by epoch
history_dict = history2.history
loss_values = history_dict['loss'] # you can change this
val_loss_values = history_dict['val_loss'] # you can also change this
epochs = range(1, len(loss_values) + 1) #acc wasn't defined before
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'orange', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model2.save("/content/drive/My Drive/Spring Deep Learning Class material /Week 3/Weather project / “ConvNet_WindRegression_Model_tiz18015.h5")

In [ ]:
# make predictions on the testing data
print("[INFO] predicting weather speed...")
preds_test = model2.predict(x_test)
plt.scatter(preds_test, y_test_3)
plt.title('sknt Test: Actual vs. Predicted')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig(result+'sknt Test.png')

In [ ]:
results = model2.evaluate(x_test, y_test_3)
results

In [ ]:
print(type(preds_test))
print(type(y_test_3))

In [ ]:
myPreds = np.ndarray.flatten(preds_test)
myPreds = pd.DataFrame(myPreds)
myPreds.reset_index(drop=True)
myPreds.rename(columns={0: "preds_sknt"}, inplace=True)
myPreds['Type'] = 'test'
#actual predictions
myActual = pd.DataFrame(np.array(y_test_3))
myActual.reset_index(drop=True)
myActual.rename(columns={0: "actual_sknt"}, inplace=True)
allResults3 = pd.concat([myActual,myPreds], axis=1)
allResults3.head()

In [ ]:
allResults3.to_csv('/content/drive/My Drive/Spring Deep Learning Class material /Week 3/Weather_project.csv')

In [ ]:
# conclusion 
# from the learning curve, we can see the model is pretty good, there is no overfitting, cause we use early stop method. However, when we see
# the scatter plot, it is not very well, which means that the model not lewarn very vell, maybe because symbols about wind speed is not very easy to learn.



Part 2 Binary Classification Model for Air Temperature

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [ ]:
target_data.head()

,tmpf,drct,sknt
valid,,,
2016-09-01 00:00:00,75.02,222.500000,5.125000
2016-09-01 01:00:00,73.94,202.857143,4.857143
2016-09-01 02:00:00,73.94,218.181818,4.272727
2016-09-01 03:00:00,73.94,204.444444,4.555556
2016-09-01 04:00:00,73.04,145.000000,2.700000


In [ ]:
# Create a binary target variable using median_house_value, above average and below average value
above_avg = target_data[target_data['tmpf']>= target_data['tmpf'].mean()]
below_avg = target_data[target_data['tmpf']< target_data['tmpf'].mean()]
above_avg['tmpf'] = 1
below_avg['tmpf'] = 0
binary_model_data = pd.concat([above_avg,below_avg], axis=0)
binary_model_data=binary_model_data.sort_index(axis = 0) 
binary_model_data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,tmpf,drct,sknt
valid,,,
2016-09-01 00:00:00,1,222.500000,5.125000
2016-09-01 01:00:00,1,202.857143,4.857143
2016-09-01 02:00:00,1,218.181818,4.272727
2016-09-01 03:00:00,1,204.444444,4.555556
2016-09-01 04:00:00,1,145.000000,2.700000
...,...,...,...
2017-08-31 19:00:00,1,320.000000,8.583333
2017-08-31 20:00:00,1,305.454545,11.000000
2017-08-31 21:00:00,1,315.833333,10.583333


In [ ]:
print("Image train dataset shape: ", x_train.shape, "Weather train dataset shape: ", train.shape)
print("Image validation dataset shape: ", x_val.shape, "Weather validation dataset shape: ", val.shape)
print("Image test dataset shape: ", x_test.shape, "Weather test dataset shape: ", test.shape)

Image train dataset shape:  (6511, 125, 145, 3) Weather train dataset shape:  (6511, 3)
Image validation dataset shape:  (1464, 125, 145, 3) Weather validation dataset shape:  (1464, 3)
Image test dataset shape:  (744, 125, 145, 3) Weather test dataset shape:  (744, 3)


In [ ]:
# combine x_train, x_val, x_test
X = np.concatenate((x_train, x_val,x_test), axis=0)
y = binary_model_data.tmpf

In [ ]:
# Randomly split X and y to train, validation (1 month) and test (1 month)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.083, random_state= 525)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.091, random_state=525)

In [ ]:
# check shape
print("Image train dataset shape: ", X_train.shape, "Weather train dataset shape: ", y_train.shape)
print("Image validation dataset shape: ", X_val.shape, "Weather validation dataset shape: ", y_val.shape)
print("Image test dataset shape: ", X_test.shape, "Weather test dataset shape: ", y_test.shape)


Image train dataset shape:  (7267, 125, 145, 3) Weather train dataset shape:  (7267,)
Image validation dataset shape:  (728, 125, 145, 3) Weather validation dataset shape:  (728,)
Image test dataset shape:  (724, 125, 145, 3) Weather test dataset shape:  (724,)


In [ ]:
# build model
from keras import models
from keras import layers
model1 = models.Sequential()
model1.add(layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', input_shape=(125,145,3))) 
model1.add(layers.MaxPooling2D(pool_size=2))
model1.add(layers.Dropout(0.3))

model1.add(layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu')) 
model1.add(layers.Dropout(0.3))

model1.add(layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model1.add(layers.MaxPooling2D(pool_size=2))
model1.add(layers.Dropout(0.3))

model1.add(layers.Flatten())
model1.add(layers.Dense(512, activation='relu'))  # maybe because too many layers.
model1.add(layers.Dropout(0.2))
model1.add(layers.Dense(512, activation='relu'))
model1.add(layers.Dropout(0.2))
model1.add(layers.Dense(256, activation='relu'))
model1.add(layers.Dropout(0.2))
model1.add(layers.Dense(128, activation='relu'))
model1.add(layers.Dropout(0.2))
model1.add(layers.Dense(1,activation='sigmoid'))








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
from keras import optimizers
model1.compile(loss='binary_crossentropy',
              optimizer = optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
es1 = EarlyStopping(monitor='val_loss', mode='min', 
                   patience=5, 
                   verbose=1)

In [ ]:
history1 = model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=500, batch_size=5,verbose=1,callbacks=[es1])




Train on 7267 samples, validate on 728 samples
Epoch 1/500





7267/7267 [==============================] - 71s 10ms/step - loss: 7.9293 - acc: 0.5025 - val_loss: 8.0150 - val_acc: 0.4973
Epoch 2/500
7267/7267 [==============================] - 54s 7ms/step - loss: 7.9350 - acc: 0.5023 - val_loss: 8.0150 - val_acc: 0.4973
Epoch 3/500
7267/7267 [==============================] - 54s 7ms/step - loss: 7.9350 - acc: 0.5023 - val_loss: 8.0150 - val_acc: 0.4973
Epoch 4/500
7267/7267 [==============================] - 54s 7ms/step - loss: 7.9350 - acc: 0.5023 - val_loss: 8.0150 - val_acc: 0.4973
Epoch 5/500
7267/7267 [==============================] - 54s 7ms/step - loss: 7.9350 - acc: 0.5023 - val_loss: 8.0150 - val_acc: 0.4973
Epoch 6/500
7267/7267 [==============================] - 54s 7ms/step - loss: 7.9350 - acc: 0.5023 - val_loss: 8.0150 - val_acc: 0.4973
Epoch 00006: early stopping


In [ ]:
# let's try to extract from history.history
x = np.array(history1.epoch)
x = max(x)
print("We should re-run for this many epochs: ", x)

We should re-run for this many epochs:  5


In [ ]:
model1.save("/content/drive/My Drive/Spring Deep Learning Class material /Week 3/Weather project / “ConvNet_TemperatureClassification_Model4_tiz18015.h5")

In [ ]:
# preidct for test 
preds_test = model1.predict(X_test)
preds_test = []
for i in preds_test:
   if i >= 0.5:
     preds_test.append(1)
   else:
     preds_test.append(0)
print (classification_report(y_test,preds_test))

In [ ]:
from sklearn.metrics import accuracy_score
test_score = accuracy_score(y_test, a)
test_score 

In [ ]:

# flatten the 1-D numpy array
myPreds = np.ndarray.flatten(preds_test)
myPreds = pd.DataFrame(myPreds)
myPreds.reset_index(drop=True)
myPreds.rename(columns={0: "preds_test"}, inplace=True)

#actual predictions
myActual = pd.DataFrame(np.array(y_test))
myActual.reset_index(drop=True)
myActual.rename(columns={0: "actual_test"}, inplace=True)

myprobability = pd.DataFrame(np.array(a))
myprobability.reset_index(drop=True)
myprobability.rename(columns={0: "probability_test"}, inplace=True)

allResults_test = pd.concat([myActual,myPreds,myprobability], axis=1)
allResults_test.head()
allResults_test.to_csv('/content/drive/My Drive/Spring Deep Learning Class material /Week 3/Weather project /Saved_model/predicted_test.csv')

In [ ]:
# flatten the 1-D numpy array
myPreds = np.ndarray.flatten(preds_train)
myPreds = pd.DataFrame(myPreds)
myPreds.reset_index(drop=True)
myPreds.rename(columns={0: "preds_train"}, inplace=True)

#actual predictions
myActual = pd.DataFrame(np.array(y_train))
myActual.reset_index(drop=True)
myActual.rename(columns={0: "actual_train"}, inplace=True)

myprobability = pd.DataFrame(np.array(b))
myprobability.reset_index(drop=True)
myprobability.rename(columns={0: "probability_train"}, inplace=True)

allResults_test = pd.concat([myActual,myPreds,myprobability], axis=1)
allResults_test.head()
allResults_test.to_csv('/content/drive/My Drive/Spring Deep Learning Class material /Week 3/Weather project /Saved_model/predicted_train.csv')
